In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gym_bandits
import gym
import math
env = gym.make("BanditTenArmedGaussian-v0")

In [9]:
env.action_space

Discrete(10)

In [21]:
def initialize_Q(num_arms):
    return np.zeros(num_arms)

In [38]:
num_rounds = 20000

# random initialize Q value which is the average reward
Q_epsilon = Q_softmax = Q_UCB = Q_TS = alpha = beta = count = sum_rewards = initialize_Q(10)

In [32]:
def epsilon_greedy(epsilon):
    
    if np.random.random() < epsilon:
        action = env.action_space.sample()
    else: 
        action = np.argmax(Q)
        
    return action

def softmax(tau):
    total = sum([math.exp(val/tau) for val in Q])
    probs = [math.exp(val/tau)/total for val in Q]
    threshold = random.random()
    cumulative_prob = 0.0
    
    for i in range(len(probs)):
        cumulative_prob += probs[i]
        if (cumulative_prob > threshold):
            return i
    return np.argmax(probs)

def UCB(iters):
    
    ucb = np.zeros(10)
    # check if consists at least 10 sample
    
    if iters < 10:
        return i
    
    else:
        for arm in range(10):
            # calculate upper bound
            upper_bound = math.sqrt((2*math.log(sum(count))) / count[arm])

            # add upper bound to the Q value
            ucb[arm] = Q_UCB[arm] + upper_bound
        
    # return the arm which has maximum value
    return (np.argmax(ucb))

In [33]:
for i in range(num_rounds):
    # Select the arm using UCB
    arm = UCB(i)
    # Get the reward
    observation, reward, done, info = env.step(arm)
    # update the count of that arm
    count[arm] += 1
    # Sum the rewards obtained from the arm
    sum_rewards[arm]+=reward
    # calculate Q value which is the average rewards of the arm
    Q_UCB[arm] = sum_rewards[arm]/count[arm]
    
print( 'The optimal arm is {}'.format(np.argmax(Q)))

The optimal arm is 0


In [39]:
def thompson_sampling(alpha,beta):
    samples = [np.random.beta(alpha[i]+1,beta[i]+1) for i in range(10)]
    return np.argmax(samples)

for i in range(num_rounds):

    arm = thompson_sampling(alpha,beta)

    observation, reward, done, info = env.step(arm)

    count[arm] += 1

    sum_rewards[arm]+=reward

    Q[arm] = sum_rewards[arm]/count[arm]
    
    # If it is a positive reward increment alpha
    if reward >0:
        alpha[arm] += 1
    # If it is a negative reward increment beta
    else:
        beta[arm] += 1

print( 'The optimal arm is {}'.format(np.argmax(Q)))

The optimal arm is 0


In [16]:
for i in range(num_rounds):

    # arm using epsilon greedy
    arm = epsilon_greedy(0.5)
    
    # arm using softmax
    arm = softmax(0.5)

    observation, reward, done, info = env.step(arm)

    count[arm] += 1

    sum_rewards[arm]+=reward

    # calculate Q value which is the average rewards of the arm
    Q[arm] = sum_rewards[arm]/count[arm]
    
print( 'The optimal arm is {}'.format(np.argmax(Q)))

The optimal arm is 4


In [18]:
Q

array([ 1.19831264,  0.33350828,  0.76601153, -0.3355797 ,  2.0204708 ,
       -0.98286558,  1.66036175, -0.47403209,  0.23711068, -0.55311812])